In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import glob
import matplotlib.pyplot as plt
import pprint
pp = pprint.PrettyPrinter(width=41,depth=6, compact=True)

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout


from sklearn.ensemble import RandomForestRegressor

Cıngıllı : Location 2 

Alibey   : Location 2,Location 1

Apa      : Location 2

Renoe    : Location 1 , Location 2

Omic     : Location 5 ,Location 4

Afta     : Location 2

Ra       : Location 1 

Solen    : Location 5 

In [65]:
path = r"D:\GES"
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    fl = pd.read_csv(filename, index_col=None, header=0,skipfooter=1,engine='python')
    fl=fl[['Tarih','Saat','Güneş']]
    fl['GES']=str(filename.split('-')[1])
    li.append(fl)

for i in range(18):
   # li[i]['Güneş']=li[i]['Güneş'].apply(lambda x: x.replace(',','.'))#.apply(lambda x: float(x))
    li[i]['Güneş']=li[i]['Güneş'].apply(lambda x: x.replace(',','.'))
    li[i]['Güneş']=li[i]['Güneş'].apply(lambda x: float(x))
    
maksili=[]
for i in range(18):
    maks=float(li[i]['Güneş'].max())
    maksili.append(maks)
    li[i]['Güneş']=li[i]['Güneş'].apply(lambda x: x/maksili[i])
    

data = pd.concat(li, axis=0, ignore_index=True)

data['Tarihsaat']=pd.to_datetime(data['Tarih']+' '+data['Saat'])
data=data.set_index('Tarihsaat')
data


,Tarih,Saat,Güneş,GES
Tarihsaat,,,,
2019-09-26 00:00:00,26.09.2019,00:00,0.000000,AFTA
2019-09-26 01:00:00,26.09.2019,01:00,0.000000,AFTA
2019-09-26 02:00:00,26.09.2019,02:00,0.000000,AFTA
2019-09-26 03:00:00,26.09.2019,03:00,0.000000,AFTA
2019-09-26 04:00:00,26.09.2019,04:00,0.000000,AFTA
...,...,...,...,...
2020-07-31 18:00:00,31.07.2020,18:00,0.104463,YAYSUN
2020-07-31 19:00:00,31.07.2020,19:00,0.008547,YAYSUN
2020-07-31 20:00:00,31.07.2020,20:00,0.000000,YAYSUN


In [66]:
wr=pd.read_csv('C:/Users\Ahmet YÜKSEL/Desktop/Algopoly Belgeler/trweather.csv')
wr=wr.rename(columns={"loc": "Yer", "forecast_epoch": "Time", "DSWRF_surface": "Rad","TMP_2.m.above.ground":"Temp"})
wr['Temp']=wr['Temp'].map(lambda x:x-273.15) #convert to celcius
locs=wr['Yer'].unique()

wr1=wr[wr['Yer']==locs[0]]
wr1=wr1.drop('Yer',axis=1)

wr2=wr[wr['Yer']==locs[1]]
wr2=wr2.drop('Yer',axis=1)

wr3=wr[wr['Yer']==locs[2]]
wr3=wr3.drop('Yer',axis=1)

wr4=wr[wr['Yer']==locs[3]]
wr4=wr4.drop('Yer',axis=1)

wr5=wr[wr['Yer']==locs[4]]
wr5=wr5.drop('Yer',axis=1)

wr6=wr[wr['Yer']==locs[5]]
wr6=wr6.drop('Yer',axis=1)

wr7=wr[wr['Yer']==locs[6]]
wr7=wr7.drop('Yer',axis=1)


In [67]:
def son_iki(df):
    iki=df.iloc[-30*24-2-7:25558-7]
    bir=df.iloc[(-62*24)-3-6:-31*24-2-6]
    
    return bir,iki

In [68]:
def wr_ayar(df):
    df1,df2=son_iki(df)
    A=df1['Rad'].values
    B=df1['Temp'].values
    C=df2['Rad'].values
    D=df2['Temp'].values
    dates=pd.date_range(start='2020-05-01',end='2020-06-01',freq='H')
    dates2=pd.date_range(start='2020-06-01',end='2020-07-01',freq='H')
    z=pd.DataFrame()
    q=pd.DataFrame()
    z['Rad']=A
    z['Temp']=B
    z=z.set_index(dates)
    q['Rad']=C
    q['Temp']=D
    q=q.set_index(dates2)
    df1=z.copy()
    df2=q.copy()
    ##fig, axs = plt.subplots(2, 2,figsize=(14,10))
    
    
    ##axs[0, 0].plot(dates, df1['Rad'])
    ##axs[0, 0].set_title('Training Radyal')
    ##axs[0, 1].plot(dates, df1['Temp'], 'tab:orange')
    ##axs[0, 1].set_title('Training Temp')
    ##axs[1, 0].plot(df2.index, df2['Rad'], 'tab:green')
    ##axs[1, 0].set_title('Test Radyal')
    ##axs[1, 1].plot(df2.index, df2['Temp'], 'tab:red')
    ##axs[1, 1].set_title('Test Temp')
   
    return df1,df2

In [69]:
wr1_train,wr1_test=wr_ayar(wr1)

wr2_train,wr2_test=wr_ayar(wr2)

wr3_train,wr3_test=wr_ayar(wr3)

wr4_train,wr4_test=wr_ayar(wr4)

wr5_train,wr5_test=wr_ayar(wr5)

wr6_train,wr6_test=wr_ayar(wr6)

wr7_train,wr7_test=wr_ayar(wr7)

In [70]:
def train_test(df):
    train_start=df.index[(-91*24)-23]
    train_stop=df.index[(-62*24)+25]
    
    test_start=df.index[(-60*24)-23]
    test_stop=df.index[(-32*24)+25]
    
    return train_start,train_stop,test_start,test_stop

In [71]:
train_starter='2020-05-01'
train_stoper='2020-06-01'
test_starter='2020-06-01'
test_stoper='2020-07-01'

In [72]:
dates=pd.date_range(start=train_starter,end=train_stoper,freq='H')
dates.shape

(745,)

In [79]:
def ges_ayar(df,starter,stoper):
    A=df['Güneş'].values
    dates=pd.date_range(start=starter,end=stoper,freq='H')
    z=pd.DataFrame()
    z['Güneş']=A
    z=z.set_index(dates)
    df=z.copy()
    #plt.figure(figsize=(15,4))
   # df['Güneş'].plot()
    return df

In [109]:
data_cin=data[data['GES']=='CINGILLI']
train_start,train_stop,test_start,test_stop=train_test(data_cin)

train_data_cin=data_cin[train_start:train_stop]
train_data_cin=ges_ayar(train_data_cin,train_starter,train_stoper)

test_data_cin=data_cin[test_start:test_stop]
test_data_cin=ges_ayar(test_data_cin,test_starter,test_stoper)


In [118]:
def split(df):
    #train=pd.concat([test,df]).drop_duplicates(keep=False)
    y_train=df['Güneş'].values

    x_train=df[['Rad', 'Temp', 'Saat', 'Gün']]

    print(y_train.shape)

    print(x_train.shape)
    
    return x_train,y_train

In [111]:
test_data_cin=data_cin[test_start:test_stop]
test_data_cin=ges_ayar(test_data_cin,test_starter,test_stoper)


In [112]:
def f_ekle(df):
    
    hour=pd.DataFrame(df.index.hour)
    day=pd.DataFrame(df.index.day)

    df['Saat']=hour.values
    df['Gün']=day.values
    return df

In [113]:
wr2_test=f_ekle(wr2_test)
train_data_cin=f_ekle(train_data_cin)


In [114]:
train_data_cin=pd.merge(train_data_cin,wr2_train,how='outer',left_index=True, right_index=True)

In [115]:
x_train,y_train=split(train_data_cin)

(745,)
(745, 4)


In [116]:
test_data_cin=pd.merge(test_data_cin,wr2_test,how='outer',left_index=True, right_index=True)

In [119]:
x_test,y_test=split(test_data_cin)

(721,)
(721, 4)


# Modeller

### Random Forest

In [121]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(x_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   14.2s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   57.4s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.0min finished


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [124]:
preds_rf=rf_random.predict(wr2_test)

predictions=wr2_test.copy()
predictions['Güneş']=preds_rf


In [128]:
real=test_data_cin['Güneş'].values

### XgBoost

In [130]:
import xgboost as xgb
from xgboost import XGBRegressor

xgb = XGBRegressor(booster='gbtree', learning_rate=0.3, n_estimators=3460,
                     max_depth=6, min_child_weight=1, subsample=1,
                     gamma=0, reg_alpha = 0.001, colsample_bytree=0.7,
                     objective='reg:squarederror', reg_lambda = 0.001,
                     scale_pos_weight=1, seed=2020)

from sklearn.model_selection import RandomizedSearchCV

param_lst = {
    'learning_rate' : [0.01, 0.1, 0.15, 0.3, 0.5],
    'n_estimators' : [100, 500, 1000, 2000, 3000],
    'max_depth' : [3, 6, 9],
    'min_child_weight' : [1, 5, 10, 20],
    'reg_alpha' : [0.001, 0.01, 0.1],
    'reg_lambda' : [0.001, 0.01, 0.1]
}

xgb_reg = RandomizedSearchCV(estimator = xgb, param_distributions = param_lst,
                              n_iter = 100, scoring = 'neg_root_mean_squared_error',
                              cv = 5)
       
xgb_search = xgb_reg.fit(x_train, y_train)

# XGB with tune hyperparameters
best_param = xgb_search.best_params_
xgb = XGBRegressor(**best_param)
xgb.fit(x_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=3,
             min_child_weight=20, missing=nan, monotone_constraints='()',
             n_estimators=500, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0.01, reg_lambda=0.001, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [133]:
preds_xgb=xgb.predict(wr2_test)

predictions_xgb=wr2_test.copy()
predictions_xgb['Güneş']=preds_xgb


In [136]:
predictions_xgb['Güneş']=predictions_xgb['Güneş'].apply(lambda x: 0 if x<0 else x)

In [138]:
preds_xgb=predictions_xgb['Güneş'].values

### Mean Squared Error

preds_rf : random forest tahmin sonucu

preds_xgb : Xgboost tahmin sonucu

In [129]:
from sklearn.metrics import mean_squared_error
mean_squared_error(real, preds_rf)

0.01699742942481815

In [139]:
mean_squared_error(real,preds_xgb)

0.017270029598464937